# Estudo de caso: Bacia do Paranapanema

## Imports

In [848]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
import hydrobr

from plotly.offline import plot
from mogestpy.quality import buwo
from mogestpy.quantity.Hydrological import SMAP
from mogestpy.quantity.Hydrological import Muskingum
from spotpy.objectivefunctions import rmse, nashsutcliffe, pbias, kge

pd.options.plotting.backend = "plotly"
pio.templates.default = "plotly_white"

## Inicialização do modelo de bacias

### Dados com os parâmetros SMAP - Buildup/Washoff - Muskingum

In [849]:
df_data = pd.read_excel('case.xlsx')

### Dados meteorológicos

In [850]:
df_prec = pd.read_excel('prec.xlsx')
df_etp = pd.read_excel('etp.xlsx')

Carrega as informações da bacia para o modelo SMAP

In [851]:
basins = []
for i in range(8):
    basin = SMAP.SMAP.Basin(AD=df_data['AD'][i],
                            Str=df_data['Sat'][i],
                            Capc=df_data['Capc'][i],
                            Crec=df_data['Crec'][i],
                            TUin=df_data['Tuin'][i],
                            EBin=df_data['Ebin'][i],
                            k2t=df_data['k2t'][i],
                            kkt=df_data['kkt'][i],
                            Ai=df_data['Ai'][i])
    basins.append(basin)

Códigos das ottobacias de nível 5 do estudo de caso

In [852]:
codes = [86492, 86493, 86494, 86495, 86496, 86497, 86498, 86499]

Criando pontos com precipitação e evapotranspiração das bacias

In [853]:
points = []
for i in range(8):
    point = SMAP.SMAP.Point(df_prec[codes[i]], df_etp[codes[i]])
    points.append(point)
    
models = []
for i in range(8):
    model = SMAP.SMAP(points[i], basins[i])
    models.append(model)

Roda bacias modeladas

In [854]:
for model in models:
    model.RunModel()

Criando DataFrames (Tabelas) de vazão incremental (cada ottobacia) e escoamento superficial direto

In [855]:
df_Qinc = pd.DataFrame()
df_Qd = pd.DataFrame()

In [856]:
for i in range(8):
    df_Qinc[codes[i]] = models[i].Q
    df_Qd[codes[i]] = models[i].Qd


Criando coluna com dados da estação de monitoramento a montante da bacia 86497 (das quais as bacias 86498 e 86499 contribuem)

In [857]:
df_Qinc['86498+86499'] = df_Qinc[86498] + df_Qinc[86499]

Carregando dados de vazão observada para a estação de monitoramento utilizada

In [858]:
df_obs = pd.read_excel('qobs.xlsx')

In [859]:
df_Qinc['Qobs'] = df_obs['Qobs']

Gráfico de vazão na estação de monitoramento 2011 - 2012

In [860]:
df_Qinc['Qobs'] = df_obs['Qobs'].replace(0, float('nan')).dropna()
fig = df_Qinc[['86498+86499', 'Qobs']].plot()

fig.update_layout(legend_title='Legenda', legend=dict(y=1.1, orientation='h', itemsizing='constant'), font_family='CMU Serif', font=dict(size=12))
fig.update_xaxes(title_text='Tempo (dias)', title_font=dict(size=12))
fig.update_yaxes(title_text='Vazão (m³/s)', title_font=dict(size=12))
fig.update_layout(
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=700,
    height=500
)

fig.show()

Calculando indicadores para o período de 2012

In [861]:

def indicator(obs, sim):
    rmse_val = rmse(obs, sim)
    pbias_val = pbias(obs, sim)
    nashsutcliffe_val = nashsutcliffe(obs, sim)
    kge_val = kge(obs, sim)

    print(f"RMSE: {rmse_val:.1f}")
    print(f"PBIAS: {pbias_val:.1f}%")
    print(f"NSE: {nashsutcliffe_val:.3f}")
    print(f"KGE: {kge_val:.3f}")

# Example usage:
indicator(df_Qinc['Qobs'][365:], df_Qinc['86498+86499'][365:])



RMSE: 25.4
PBIAS: -4.8%
NSE: 0.650
KGE: 0.812


Verificação dos valores das parcelas de escoamento base e escoamento superficial direto

In [862]:
df_test = pd.DataFrame()

Verificação da vazão superficial direta e de base do modelo SMAP

In [863]:
df_test['Qb'] = models[0].Qb
df_test['Qd'] = models[0].Qd
df_test['Qt'] = models[0].Q

fig = df_test.plot()
fig.update_layout(legend_title='Legenda', legend=dict(y=1.1, orientation='h', itemsizing='constant'), font_family='CMU Serif', font=dict(size=12))
fig.update_xaxes(title_text='Tempo (dias)', title_font=dict(size=12))
fig.update_yaxes(title_text='Vazão (m³/s)', title_font=dict(size=12))
fig.update_layout(
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=800,
    height=600
)

fig.show()


Configurando o modelo de acumulo e lavagem em bacias (Buildup/Washoff)

In [864]:
loadgens = []

for i in range(8):
    loadgen = buwo.BuildUpWashoff(landuse_name='',
                                  Bmax=df_data['Bmax'][i],
                                  Nb=df_data['Nb'][i],
                                  Kb=df_data['Kb'][i],
                                  threshold_flow=0.02,
                                  Nw=df_data['Nw'][i],
                                  Kw=df_data['Kw'][i],
                                  BuMethod=2,
                                  WoMethod=1,
                                  timestep_h=24,
                                  initial_buildup=1,
                                  area=df_data['AD'][i],
                                  area_fraction=1,
                                  surface_flow=df_Qd[codes[i]]/df_data['AD'][i]*86.4)
    loadgens.append(loadgen)

Convertendo dados de vazão de (**m³/s**) para (**mm/dia**)

In [865]:
df_Qmm = pd.DataFrame()

for i in range(8):
    df_Qmm[codes[i]] = df_Qd[codes[i]]/df_data['AD'][i]*86.4

Gráfico de Escoamento Superficial Direto para toda a região do estudo

In [866]:
fig = df_Qmm.plot()
fig.update_layout(
    xaxis_title='Tempo (dias)',
    yaxis_title='Vazão (mm/dia)',
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=700,
    height=500
)
fig.show()


In [867]:
df_Qmm.to_excel('Vazao_mm.xlsx')

In [868]:
for loadgen in loadgens:
    loadgen.Process(verbose=False)

In [869]:
df_Wo = pd.DataFrame()
for i in range(8):
    df_Wo[codes[i]] = loadgens[i].Washoff

Gráfico de Washoff (Lavagem) em massa DBO

In [870]:
fig = df_Wo.plot()
fig.update_layout(
    xaxis_title='Tempo (dias)',
    yaxis_title='Carga (kg/dia)',
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=800,
    height=600
)
fig.show()


In [871]:
try:
    df_Wo.to_excel('Cargas.xlsx')
    print("File saved successfully!")
except Exception as e:
    print(f"Error saving file: {e}")


File saved successfully!


In [872]:
df_WoC = pd.DataFrame()
for i in range(8):
    df_WoC[codes[i]] = df_Wo[codes[i]]/df_Qinc[codes[i]]/86.4

Polutograma (mg/L) DBO

In [873]:
fig = df_WoC[365:].plot()
fig.update_layout(
    xaxis_title='Tempo (dias)',
    yaxis_title='Carga (mg/L)',
    legend_title='Legenda',
    legend=dict(y=1.1, orientation='h', itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=800,
    height=600
)
fig.show()


In [874]:
df_WoC.to_excel('Cargas_mg-L.xlsx')

In [875]:
df_Bu = pd.DataFrame()  

for i in range(8):
    df_Bu[codes[i]] = loadgens[i].BuildUp

In [876]:
df_Bu_ton = df_Bu.div(1e3) # conversão para ton

### Gráfico de Buildup para o ano de 2012

x = tempo (dias); x0 = 01/01/2012

y = Buildup (kg/dia)

In [877]:
fig = df_Bu_ton[:365].plot.bar()

fig.update_yaxes(title_text='Carga (ton/dia)', secondary_y=False)
fig.update_layout(xaxis_title='Tempo (dias)', font=dict(size=12), legend=dict(y=1.1, orientation='h', itemsizing='constant'))
fig.update_layout(
    font=dict(size=12),
    width=1000,
    height=600
)

fig.show()

In [878]:
import locale
from plotly.subplots import make_subplots

# set locale to PT-BR
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

df_estrela = pd.DataFrame()
df_estrela['86498+86499'] = df_Bu_ton[86498] + df_Bu_ton[86499]
df_estrela = df_estrela.iloc[:365]
df_estrela['subtotal'] = df_estrela['86498+86499'].cumsum()
df_estrela.index = pd.date_range(start='2012-01-01', periods=len(df_estrela), freq='D')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=df_estrela.index, y=df_estrela['86498+86499'], name='Carga (ton/dia)', marker_color='blue'), secondary_y=False)
fig.add_trace(go.Scatter(x=df_estrela.index, y=df_estrela['subtotal'], name='Carga Acumulada (ton)', line=dict(color='red', width=2)), secondary_y=True)

fig.update_layout(title='Ponto de Controle - Montante da Ottobacia 86497', xaxis_title='Tempo (dias)', font=dict(size=12), legend=dict(y=1.1, orientation='h', itemsizing='constant'))
fig.update_yaxes(title_text='Carga (ton/dia)', secondary_y=False)
fig.update_yaxes(title_text='Carga Acumulada (ton)', secondary_y=True)
fig.update_layout(
    font=dict(size=12),
    width=800,
    height=600
)

fig.show()



In [879]:
df_Bu.to_excel('BuildUp.xlsx')

PermissionError: [Errno 13] Permission denied: 'BuildUp.xlsx'

Download das estações, se `downloaded = True` será utilizado a planilha

In [ ]:
stations = [64214200,
64214100,
64214050,
64214000,
64205000,
64200000,
64198010,
64198002,
64198000,
64191000,
64190800,
64190000,
64185000,
64183000,
64182000,
64181000,
64150000,
64149000,
64135001,
64135000,
64120100,
64120000,
64119500,
64119000,
64115000,
64114000,
64113000,
64112000,
64105800,
64105000,
64101000,
64100000,
64099000,
64098000,
64096000,
64095000,
64082000,
64081000,
64080000,
64075200,
64075100,
64075000,
64074800,
64066000,
64065000,
64040000,
64035000,
64030000,
64015000,
64014800,
64013000,
64012500,
64010000,
64007000,
64005000]
downloaded = True
if not downloaded:
    data_stations = hydrobr.get_data.ANA.flow_data(stations)
else:
    data_stations = pd.read_excel('flow_stations.xlsx', index_col=0, parse_dates=True)


In [ ]:
disponib = hydrobr.Plot.gantt(data_stations)

Diagrama de disponibilidade de dados

In [ ]:
if not downloaded:
    disponib.update_layout(
        autosize=False,
        width=1000,
        height=500,
        xaxis_title = 'Year',
        yaxis_title = 'Station Code',
        font=dict(family="Courier New, monospace", size=12))

In [ ]:
data_stations = data_stations.loc['2012-01-01':'2012-12-31']
data_stations.dropna(axis=1, how='all', inplace=True)
data_stations.to_excel('flow_stations.xlsx')


In [ ]:
disponib = hydrobr.Plot.gantt(data_stations)
disponib.update_layout(
    autosize=False,
    width=1000,
    height=500,
    xaxis_title = 'Year',
    yaxis_title = 'Station Code',
    font=dict(family="Courier New, monospace", size=12))

In [ ]:
codes2 = ['86492', '86496', '86495']

In [ ]:
catchment_station = {
    '86492': '64198000',
    '86496': '64119000',
    '86495': '64135000'
}

In [ ]:
stations2 = ['64198000', '64119000', '64135000']

In [ ]:
catchment_area = {
    '86492': 721.3625,
    '86496': 533.07,
    '86495': 622.855
}

In [ ]:
calibrated = True

In [ ]:
if not calibrated:
    basins2 = []
    for i in catchment_area.values():
        basin = SMAP.SMAP.Basin(AD=i)
        basins2.append(basin)   
else:
    df_data2 = pd.read_excel('case_calib2.xlsx')
    
    basins2 = []
    for i in range(3):
        basin = SMAP.SMAP.Basin(AD=df_data2['AD'][i],
                                Str=df_data2['Sat'][i],
                                Capc=df_data2['Capc'][i],
                                Crec=df_data2['Crec'][i],
                                TUin=df_data2['TUin'][i],
                                EBin=df_data2['EBin'][i],
                                k2t=df_data2['k2t'][i],
                                kkt=df_data2['kkt'][i],
                                Ai=df_data2['Ai'][i])
        basins2.append(basin)

In [ ]:
basins2[0].kkt = 70


In [ ]:
points2 = []
for station in catchment_station.keys():
    point = SMAP.SMAP.Point(df_prec[int(station)][365:].to_list(), df_etp[int(station)][365:].to_list())
    points2.append(point)

In [ ]:
models2 = []

pcof = [1.4, 1.6, 1]

for i in range(3):
    for j in range(points2[i].n):
        points2[i].P[j] = points2[i].P[j] * pcof[i]
    
    model = SMAP.SMAP(points2[i], basins2[i])
    models2.append(model)

In [ ]:
if not calibrated:
    calibs = []
    for i in range(3):
        calibs.append(models2[i].Calibrate(data_stations[stations2[i]],
                             objective_function='rmse',
                             optimization_engine = 'de'
                             )                   
                      )
else:
    for i in range(3):
        models2[i].RunModel() 

In [ ]:
show_params = True

if show_params:
    for i in range(3):
        print(codes2[i])
        print(models2[i].Basin)
        print()

In [ ]:
df_Q2 = pd.DataFrame()
for i in range(3):
    df_Q2[codes2[i]] = models2[i].Q

In [ ]:
for i in range(3):
    print(f'Estação {codes2[i]}')
    print()
    print('Perído total')
    indicator(data_stations[stations2[i]], models2[i].Q)
    print('Período de 180 dias')
    indicator(data_stations[stations2[i]][:181], models2[i].Q[:181])
    print()

In [ ]:
for i in range(3):
    df_Q2[f'{codes2[i]}_Obs'] = data_stations[stations2[i]].values

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)

for i in range(3):
    fig.add_trace(go.Scatter(x=df_Q2.index, y=df_Q2[df_Q2.columns[i]], name=f'{df_Q2.columns[i]}_Sim'), row=i+1, col=1)
    fig.add_trace(go.Scatter(x=df_Q2.index, y=df_Q2[f'{df_Q2.columns[i]}_Obs'], name=f'{df_Q2.columns[i]}_Obs'), row=i+1, col=1)

fig.update_layout(legend_title='Legenda', legend=dict(y=1.1, itemsizing='constant'), font_family='CMU Serif', font=dict(size=12))
fig.update_xaxes(title_text='Tempo (dias)', title_font=dict(size=12))
fig.update_yaxes(title_text='Vazão (m³/s)', title_font=dict(size=12))
fig.update_layout(
    legend_title='Legenda',
    legend=dict(y=1.1, itemsizing='constant'),
    font_family='CMU Serif',
    font=dict(size=12),
    width=700,
    height=800
)

# Add titles to each subplot
fig.update_layout(
    title={
        'text': "Sub-bacia 86492",
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font=dict(size=14),
    annotations=[
        dict(
            text="Sub-bacia 86496",
            x=0.5,
            y=0.65,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=14)
        ),
        dict(
            text="Sub-bacia 86495",
            x=0.5,
            y=0.35,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=14)
        )
    ]
)


fig.show()

In [ ]:
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)

for i, col in enumerate(df_Q2.columns[:3]):
    sorted_vals = np.sort(df_Q2[col])[::-1]
    ranks = np.arange(1, len(sorted_vals)+1)
    fig.add_trace(go.Scatter(x=ranks, y=sorted_vals, name=f'{col}_Sim'), row=i+1, col=1)

    sorted_vals_obs = np.sort(df_Q2[f'{col}_Obs'])[::-1]
    ranks_obs = np.arange(1, len(sorted_vals_obs)+1)
    fig.add_trace(go.Scatter(x=ranks_obs, y=sorted_vals_obs, name=f'{col}_Obs'), row=i+1, col=1)

    fig.update_xaxes(row=i+1, col=1)
    fig.update_yaxes(title_text='Vazão (m³/s)', row=i+1, col=1)
    fig.update_layout(
        legend_title='Legenda',
        legend=dict(y=1.1, orientation='h', itemsizing='constant'),
        font_family='CMU Serif',
        font=dict(size=12),
        width=700,
        height=800
    )

fig.show()


### Calibração das bacias 83 e 84

Estratégia: utilização das vazões afluentes do reservatório de Jurumirim

In [ ]:
df_jurumirim = pd.read_excel('Qjurumirim.xlsx')

In [ ]:
columns = [86492, 86493, 86494, 86495, 86496, 86497, 86498, 86499]
missing_columns = [col for col in columns if col not in df_Qinc.columns]
if missing_columns:
    print(f"One or more columns are missing in df_Qinc: {missing_columns}")
else:
    df_Qinc['Total'] = df_Qinc[columns].sum(axis=1)


In [ ]:
df_total = pd.DataFrame()
df_total['SMAP'] = df_Qinc['Total'][365:].values
df_total['Jurumirim'] = df_jurumirim['Qafl'].values
df_total['Jurumirim_nat'] = df_jurumirim['Qnat'].values
df_total['SMAP_musk'] = Muskingum.Muskingum.downstream_routing(df_total.SMAP.values, 1.6, .25, 1)

In [ ]:
df_calib_t = pd.DataFrame()
df_calib_t['Q'] = (df_total['Jurumirim'].values - df_Qinc[86492][365:].values 
                     - df_Qinc[86495][365:].values
                     - df_Qinc[86496][365:].values - df_Qinc[86497][365:].values
                     - df_Qinc[86498][365:].values - df_Qinc[86499][365:].values)
df_calib_t['Q2'] = (df_total['Jurumirim'].values - df_Qinc[86499][365:].values - df_Qinc[86498][365:].values )

In [ ]:
dif = df_total['SMAP'].sum() - df_total['Jurumirim'].sum()
print(f'{dif:.1f}')

Hidrograma (2012) no Reservatório Jurumirim

In [ ]:
df_total.plot()

In [ ]:
indicator(df_total['Jurumirim'], df_total['SMAP'])

Curva de permanência para Jurumirim

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

sorted_vals = np.sort(df_total['SMAP'])[::-1]
ranks = np.arange(1, len(sorted_vals)+1)
fig.add_trace(go.Scatter(x=ranks, y=sorted_vals, name='SMAP'))

sorted_vals_jurumirim = np.sort(df_total['Jurumirim'])[::-1]
ranks_jurumirim = np.arange(1, len(sorted_vals_jurumirim)+1)
fig.add_trace(go.Scatter(x=ranks_jurumirim, y=sorted_vals_jurumirim, name='Jurumirim'))

fig.update_xaxes(title='Rank')
fig.update_yaxes(title='Flow')
fig.update_layout(title='Duration Curve')

fig.show()
